In [12]:
!pip install transformers torch pandas
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [13]:
import csv
with open('Resources\sentiment_movies.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
# df = pd.read_csv('movie_reviews.csv', error_bad_lines=False)

In [15]:
# df = pd.read_csv('movie_reviews.csv', delimiter=',', quotechar='"')

In [16]:
# df = pd.read_csv('movie_reviews.csv', engine='python')

In [17]:
# Specify the delimiter if it's not a comma
df = pd.read_csv('Resources\sentiment_movies.csv', delimiter=',')

# If the file uses a different quote character, specify it
df = pd.read_csv('Resources\sentiment_movies.csv', quotechar='"')

# If the file has a mix of delimiters or special characters, you might need to preprocess the file
# or use the on_bad_lines parameter to skip problematic lines
df = pd.read_csv('Resources\sentiment_movies.csv', on_bad_lines='skip')

In [18]:
print(df.columns)

Index(['Unnamed: 0', 'movie_id', 'popularity', 'title', 'vote_average',
       'vote_count', 'cleaned_reviews', 'type_Action', 'type_Adventure',
       'type_Animation', 'type_Comedy', 'type_Crime', 'type_Documentary',
       'type_Drama', 'type_Family', 'type_Fantasy', 'type_History',
       'type_Horror', 'type_Music', 'type_Mystery', 'type_Romance',
       'type_Science Fiction', 'type_TV Movie', 'type_Thriller', 'type_War',
       'type_Western', 'review_neg', 'review_neu', 'review_pos',
       'review_compound', 'clean_overview', 'overview_neg', 'overview_neu',
       'overview_pos', 'overview_compound', 'release_date', 'release_year'],
      dtype='object')


In [19]:
# inputs = tokenizer(df['reviews'].tolist(), padding=True, truncation=True, return_tensors="pt")

In [20]:
print(df.head())

   Unnamed: 0  movie_id  popularity                            title  \
0           0    693134    4534.956                   Dune: Part Two   
1           1   1011985    3214.314                  Kung Fu Panda 4   
2           2    823464    2297.340  Godzilla x Kong: The New Empire   
3           3    359410    1134.726                       Road House   
4           4    634492     921.140                       Madame Web   

   vote_average  vote_count  \
0         8.311        2707   
1         7.066         897   
2         6.700         575   
3         7.066        1379   
4         5.676        1046   

                                     cleaned_reviews  type_Action  \
0  dune part two surpasses even highest expectati...        False   
1  kung fu panda best kung fu panda film even bes...         True   
2  godzilla kong new empire version previous inst...         True   
3  poor scenario story add ending also convincing...         True   
4  start heavily pregnant woman dee

In [22]:
# # Load the tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# # Load the movie reviews
# df = pd.read_csv('Resources\sentiment_movies.csv')

# # Tokenize the reviews
# inputs = tokenizer(df['cleaned_reviews'].tolist(), padding=True, truncation=True, return_tensors="pt")

# # Perform sentiment analysis
# outputs = model(**inputs)

# # Get the predicted sentiment scores
# sentiment_scores = outputs.logits.argmax(dim=-1)

# # Convert the scores to a list
# sentiment_scores_list = sentiment_scores.tolist()

# # Add the sentiment scores to the DataFrame
# df['sentiment_score'] = sentiment_scores_list

# # Convert the scores to sentiment labels
# sentiment_labels = ['neg', 'neu', 'pos']
# df['sentiment_label'] = [sentiment_labels[score] for score in df['sentiment_score']]

# # Print the sentiment labels
# print(df['sentiment_label'])

In [27]:
# import torch
# from torch.utils.data import DataLoader
# from transformers import AutoModelForSequenceClassification, AutoTokenizer

# # Assuming you have a DataFrame `df` with your data and a model `model`
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# # Prepare your DataLoader
# # Assuming `df` is your DataFrame and `tokenizer` is your tokenizer
# # You'll need to convert your DataFrame to a dataset and then to a DataLoader
# # This is a simplified example, you'll need to adapt it to your specific dataset
# dataset = YourDataset(df, tokenizer) # YourDataset should be a custom class you define
# dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# # Set up optimizer and loss function
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
# loss_function = torch.nn.CrossEntropyLoss()

# # Gradient accumulation parameters
# accumulation_steps = 4 # Accumulate gradients over 4 steps

# # Training loop
# model.train()
# for epoch in range(num_epochs):
#     for step, batch in enumerate(dataloader):
#         # Forward pass
#         inputs, labels = batch
#         outputs = model(**inputs)
#         loss = loss_function(outputs.logits, labels)
        
#         # Backward pass
#         loss.backward()
        
#         # Gradient accumulation
#         if (step + 1) % accumulation_steps == 0:
#             # Update weights
#             optimizer.step()
#             # Zero the gradients
#             optimizer.zero_grad()

#     print(f"Epoch {epoch+1}/{num_epochs} completed.")

In [28]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
        return {key: torch.tensor(val) for key, val in encoding.items()}, torch.tensor(label, dtype=torch.long)

In [29]:
# Assuming `df` is your DataFrame with columns 'text' and 'label'
texts = df['text'].tolist()
labels = df['label'].tolist()

# Initialize your custom dataset
dataset = TextDataset(texts, labels, tokenizer)

# Now you can create your DataLoader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

KeyError: 'text'

In [30]:
print(df.columns)

Index(['Unnamed: 0', 'movie_id', 'popularity', 'title', 'vote_average',
       'vote_count', 'cleaned_reviews', 'type_Action', 'type_Adventure',
       'type_Animation', 'type_Comedy', 'type_Crime', 'type_Documentary',
       'type_Drama', 'type_Family', 'type_Fantasy', 'type_History',
       'type_Horror', 'type_Music', 'type_Mystery', 'type_Romance',
       'type_Science Fiction', 'type_TV Movie', 'type_Thriller', 'type_War',
       'type_Western', 'review_neg', 'review_neu', 'review_pos',
       'review_compound', 'clean_overview', 'overview_neg', 'overview_neu',
       'overview_pos', 'overview_compound', 'release_date', 'release_year'],
      dtype='object')


In [36]:
texts = df['cleaned_reviews'].tolist()
labels = df['vote_average'].tolist()

In [37]:
# Assuming `df` is your DataFrame with columns 'cleaned_reviews' and 'vote_average'
texts = df['cleaned_reviews'].tolist()
labels = df['vote_average'].tolist()

# Initialize your custom dataset
dataset = TextDataset(texts, labels, tokenizer)

# Create your DataLoader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [38]:
model.train()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [39]:
# Assuming `model`, `dataloader`, `optimizer`, and `loss_function` are already defined
num_epochs = 3 # Number of epochs to train for
accumulation_steps = 4 # Number of steps to accumulate gradients

for epoch in range(num_epochs):
    for step, batch in enumerate(dataloader):
        inputs, labels = batch
        outputs = model(**inputs)
        loss = loss_function(outputs.logits, labels)
        
        # Normalize loss to account for batch accumulation
        loss = loss / accumulation_steps
        
        # Backward pass
        loss.backward()
        
        # Update weights
        if (step + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

    print(f"Epoch {epoch+1}/{num_epochs} completed.")

C:\Users\Sean\AppData\Local\Temp\ipykernel_28748\4064501118.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val) for key, val in encoding.items()}, torch.tensor(label, dtype=torch.long)


ValueError: too many values to unpack (expected 3)

In [40]:
for epoch in range(num_epochs):
    for step, batch in enumerate(dataloader):
        inputs, labels = batch
        # Adjusted to unpack the correct number of values
        outputs = model(**inputs)
        # Assuming `outputs` is a tuple with logits as the first element
        logits = outputs[0]
        loss = loss_function(logits, labels)
        
        # Normalize loss to account for batch accumulation
        loss = loss / accumulation_steps
        
        # Backward pass
        loss.backward()
        
        # Update weights
        if (step + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

    print(f"Epoch {epoch+1}/{num_epochs} completed.")

C:\Users\Sean\AppData\Local\Temp\ipykernel_28748\4064501118.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val) for key, val in encoding.items()}, torch.tensor(label, dtype=torch.long)


ValueError: too many values to unpack (expected 3)

In [41]:
# Assuming `model` is your DistilBERT model for sequence classification
# and `inputs` is a dictionary containing 'input_ids' and 'attention_mask'
outputs = model(**inputs)

# Correctly unpack the outputs based on what the model returns
# For DistilBERT, the forward method typically returns a tuple with logits and hidden states
logits, hidden_states = outputs

# Now you can use `logits` and `hidden_states` as needed

ValueError: too many values to unpack (expected 3)

In [42]:
for epoch in range(num_epochs):
    for step, batch in enumerate(dataloader):
        inputs, labels = batch
        # Adjusted to unpack the correct number of values
        outputs = model(**inputs)
        # Assuming `outputs` is a tuple with logits as the first element
        logits = outputs[0]
        loss = loss_function(logits, labels)
        
        # Normalize loss to account for batch accumulation
        loss = loss / accumulation_steps
        
        # Backward pass
        loss.backward()
        
        # Update weights
        if (step + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

    print(f"Epoch {epoch+1}/{num_epochs} completed.")

C:\Users\Sean\AppData\Local\Temp\ipykernel_28748\4064501118.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val) for key, val in encoding.items()}, torch.tensor(label, dtype=torch.long)


ValueError: too many values to unpack (expected 3)

In [43]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(train_df=data_train,
                                                                      text_column='Reviews',
                                                                      label_columns='Sentiment',
                                                                      val_df=data_test,
                                                                      maxlen=400,
                                                                      preprocess_mode='distilbert')

NameError: name 'text' is not defined

In [45]:
pip install ktrain

     ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
     --------------------------------------- 0.0/25.3 MB 222.6 kB/s eta 0:01:54
     --------------------------------------- 0.0/25.3 MB 222.6 kB/s eta 0:01:54
     --------------------------------------- 0.0/25.3 MB 146.3 kB/s eta 0:02:53
     --------------------------------------- 0.0/25.3 MB 146.3 kB/s eta 0:02:53
     --------------------------------------- 0.0/25.3 MB 146.3 kB/s eta 0:02:53
     --------------------------------------- 0.0/25.3 MB 115.9 kB/s eta 0:03:39
     --------------------------------------- 0.1/25.3 MB 156.1 kB/s eta 0:02:42
     --------------------------------------- 0.1/25.3 MB 199.7 kB/s eta 0:02:07
     --------------------------------------- 0.1/25.3 MB 201.8 kB/s eta 0:02:06
     --------------------------------------- 0.1/25.3 MB 218.8 kB/s eta 0:01:56
     --------------------------------------- 0.1/25.3 MB

In [46]:
import ktrain
from ktrain import text

In [ ]:
pip uninstall tensorflow